In [1]:
import cv2
import glob
import torch
import numpy as np
import glob
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
import torchvision
import torch.nn as nn
import math
import tqdm
from copy import deepcopy
from torch.optim.lr_scheduler import _LRScheduler
from bs4 import BeautifulSoup
from detectors import FaceBoxes
from custom_functions import *
device = 'cuda'

In [2]:
#Defining our input and label from xml file using predefined functions
ann_pathes = glob.glob('./train/annotations/*.xml')
whole_class = []
for ann_path in ann_pathes:
    cur_class = find_word2vec_read_xml(ann_path)
    whole_class += cur_class
word2vec = {}
for i,j in enumerate(set(whole_class)):
    word2vec[j] = i
dataset = {}
numbering = 0
base_path = './train/img/'
for ann_path in ann_pathes:
    file_name, whole_class, whole_bbox, _= read_xml(ann_path,word2vec)
    img_path = base_path + file_name
    for i,j in zip(whole_class, whole_bbox):
        dataset[numbering] = [img_path, i, j]
        numbering += 1

X = []
y = []
for data in dataset.values():
    X.append([data[0],data[2]])
    y.append(data[1])

In [3]:
#Split dataset to train and test set
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, stratify = y)
#Split trainset to train and validation set
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size = 0.2, stratify = y_train)
#Initialize dataset using custom_dataset class
trainset = face_dataset(X_train, y_train, True)
valset = face_dataset(X_val, y_val, False)
testset = face_dataset(X_test, y_test, False)
#Define Batch_size
batch_size = 64
#Initialize dataloader using custom_datasets
trainloader = DataLoader(trainset, batch_size = batch_size, shuffle = True)
valloader = DataLoader(valset, batch_size = batch_size, shuffle = True) 
testloader = DataLoader(testset, batch_size = batch_size, shuffle = True)


In [4]:
#Define model and move to the device
num_classes = 5
model = EmotionClassifier(num_classes)
model.to(device)
#Define hyperparameters,optimizer,loss_function,scheduler
optimizer = torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.9)
scheduler = CosineAnnealingWarmupRestarts(optimizer, first_cycle_steps=30, cycle_mult=1.0, max_lr=0.001, min_lr=0, warmup_steps=5, gamma=1.0)
criteria = nn.CrossEntropyLoss()
epochs = 100
#Train our model using train,validation set
best_accuracy, best_model, loss_curve = train(model,trainloader,valloader,optimizer, scheduler, criteria, 70, epochs)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

97th epoch 23 iteration loss avg : 0.29007505578920245
0.7795275590551181


0it [00:00, ?it/s]

98th epoch 23 iteration loss avg : 0.30834818072617054
0.7742782152230971


0it [00:00, ?it/s]

99th epoch 23 iteration loss avg : 0.2537654915358871
0.7926509186351706


In [6]:
#Save classifier model
torch.save(best_model.state_dict(),'./best_model.pt')

In [69]:
#Load the pretrained weights
num_classes = 5
best_model = EmotionClassifier(num_classes)
best_model.load_state_dict(torch.load('./best_model.pt'))
best_model.to(device)


EmotionClassifier(
  (base_model): Sequential(
    (0): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=Fal

In [5]:
#Test our best_model with testset
print('Test Accuracy :',test(best_model,testloader))

Test Accuracy : 0.8160377358490566


In [8]:
#Inference for the test dataset
#Define our face detector model
face_DET = FaceBoxes(device = 'cuda')
test_transform = transforms.Compose([transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
#Define vec2word
vec2word = {}
for i,j in word2vec.items():
    vec2word[j] = i
best_model.eval()
#Load test image pathes
img_pathes = glob.glob('./test/img/*.jpg')
for img_path in img_pathes:
    img = cv2.imread(img_path)
    w,h,_ = img.shape
    bboxes = face_DET.detect_faces(img)
    #Go through detected box and draw it to our given image
    for bbox in bboxes:
        box = list(map(int,bbox[:4]))
        score = bbox[-1]
        test_face = image_reader(img_path, box, False, test_transform)
        test_face = test_face.to(device).unsqueeze(0)
        result = best_model(test_face).argmax()
        word = vec2word[int(result)]
        img = cv2.rectangle(img,(box[0],box[1]),(box[2],box[3]),(0,0,255))
        img = cv2.putText(img,word,(box[0],box[3]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255))
    #Show the image if you want
    cv2.imshow('Image',img)
    #Write the result to the result file
    cv2.imwrite('./result/'+img_path.split('\\')[-1],img)
    k = cv2.waitKey(1000)
    if k == 27:
        break
cv2.destroyAllWindows()

[FaceBoxes] loading with cuda
[FaceBoxes] finished loading (0.0939 sec)
